In [ ]:
#### jefan 
#### begun: Dec 29 2016, updated: Jul 15 2018
#### analysis pipeline for "Results: Visual production training"

In [ ]:
from __future__ import division
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
np = numpy
plt = pyplot
import pandas as pd
import json
import os, sys
import cPickle
import random
import scipy.stats as stats
import seaborn as sns

from IPython.core.pylabtools import figsize, getfigs

%matplotlib inline
from pylab import *
from numpy import *

## import custom experiment analysis helpers
import experiment_analysis_helpers as ah

import warnings
warnings.filterwarnings('ignore')

#### set up paths 

In [ ]:
## define data directory (where you put downloaded data)
data_dir = './data'

## modify python path in order to load in useful variables from current directory
CURR_DIR = os.getcwd()
if os.path.join(CURR_DIR) not in sys.path:
    sys.path.append(CURR_DIR) 

In [ ]:
## read in experiment data CSV
## Note:
## "TITC" = "Trained Item, Trained Category" = "Trained"
## "UITC" = "Untrained Item, Trained Category" = "Near"
## "UIUC" = "Untrained Item, Untrained Category" = "Far"
X = pd.read_csv(os.path.join(data_dir,'Pooled_Drawing_ExperimentData1.csv'))
X.columns = ['sID','trial','phase','cluster','obj','target','rank','numStrokes','trialDuration','TITC','UITC','UIUC','expid','SS']

In [ ]:
# compute Rankdiffs (change in target rank as consequence of training)
reload(ah)
print 'Computing post-pre differences in target rank ... '
Rankdifftitc,Rankdiffuitc,Rankdiffuiuc,Preranktitc, \
Prerankuitc,Prerankuiuc,Postranktitc,Postrankuiuc, \
Postrankuiuc,wid,exp = ah.computeRankdiff(X,'rank')

In [ ]:
# Compute overall recognition accuracy 
top1 = sum(X['rank']==1)/len(X)
top4 = sum(X['rank']<5)/len(X)
top8 = sum(X['rank']<9)/len(X)

print('Top 1 accuracy: {}'.format(np.round(top1,3)))
print('Top 4 accuracy: {}'.format(np.round(top4,3)))
print('Top 8 accuracy: {} '.format(np.round(top8,3)))

In [ ]:
### Descriptive statistics for rank measure

## Any difference between conditions in pre-test? 
Preranktitc_subj = ah.getSubjMean(Preranktitc)
Prerankuitc_subj = ah.getSubjMean(Prerankuitc)
Prerankuiuc_subj = ah.getSubjMean(Prerankuiuc)

print '###### Mean pretest target rank by condition ######'
print 'Trained: ' + str(np.mean(Preranktitc_subj)) + '  sem: '  + str(ah.compute_sem(Preranktitc_subj))
print 'Near: ' + str(np.mean(Prerankuitc_subj)) + '  sem: '  + str(ah.compute_sem(Prerankuitc_subj)) 
print 'Far: ' + str(np.mean(Prerankuiuc_subj)) + '  sem: '  + str(ah.compute_sem(Prerankuiuc_subj))

print ''
# Average rank in pre-test for each condition, broken down by cue type
print '###### Average rank in pre-test for each condition, broken down by cue type (image, verbal) ######'
print 'Trained: ', np.mean(Preranktitc_subj[exp==0]),np.mean(Preranktitc_subj[exp==1])
print 'Near: ', np.mean(Prerankuitc_subj[exp==0]),np.mean(Prerankuitc_subj[exp==1])
print 'Far: ', np.mean(Prerankuiuc_subj[exp==0]),np.mean(Prerankuiuc_subj[exp==1])
print ''
# Mean and SD in pre-test for each cue-type
print '###### Mean and SD in pre-test for each cue-type ######'
all_image = np.hstack((Preranktitc_subj[exp==0],Prerankuitc_subj[exp==0],Prerankuiuc_subj[exp==0]))
all_verbal = np.hstack((Preranktitc_subj[exp==1],Prerankuitc_subj[exp==1],Prerankuiuc_subj[exp==1]))
print 'Image: ', np.mean(all_image),np.std(all_image)
print 'Verbal: ', np.mean(all_verbal),np.std(all_verbal)

## Check similarity of variances across conditions

# subsample UIUC ("Far") in order to match # observations in TITC ("Trained") and UITC ("Near") conditions
Rankdiffuiuc_sub = []
Prerankuiuc_sub = []
for i in range(len(Rankdiffuiuc)):
    inds = np.random.RandomState(i).choice(range(8),4,replace=False)    
    if i == 0:
        Rankdiffuiuc_sub = Rankdiffuiuc[i][inds]
        Prerankuiuc_sub = Prerankuiuc[i][inds]        
    else:
        Rankdiffuiuc_sub = np.vstack((Rankdiffuiuc_sub,Rankdiffuiuc[i][inds]))
        Prerankuiuc_sub = np.vstack((Prerankuiuc_sub,Prerankuiuc[i][inds]))        
        
Rankdiffuiuc_subj_sub = Rankdiffuiuc_sub.mean(1)   
Prerankuiuc_subj_sub = Prerankuiuc_sub.mean(1)   

print ' '
print '###### SD of pretest rank by condition ######'
print 'Trained: ', np.round(np.std(Preranktitc_subj),2),'  Near: ', np.round(np.std(Prerankuitc_subj),2), '  Far (subsampled): ',np.round(np.std(Prerankuiuc_subj_sub),2)

In [ ]:
## trial duration 
print 'trial duration: mean = ' + str(np.mean(X['trialDuration'])) + \
'  sem = ' + str(ah.compute_sem(X['trialDuration'])) + \
'  SD =  ' + str(np.std(X['trialDuration']))
print '95% CI: ',  np.percentile(X['trialDuration'],2.5), np.percentile(X['trialDuration'],97.5)


In [ ]:
## get training timecourse
Ranktrainmat, Traintrialmat, Ranktraintimecourse, \
Top32timecourse, Top16timecourse, Top8timecourse, \
Top4timecourse, Top1timecourse = ah.getTrainingTimecourse(X)
from matplotlib.backends.backend_pdf import PdfPages

## plot training timecourse
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')
fig = plt.figure(figsize(4,8))
x = np.arange(1,6,1)
y_raw = np.mean(np.transpose(Ranktraintimecourse),1)
subjmean = np.mean(np.transpose(Ranktraintimecourse),0)
grandmean = np.mean(subjmean)
y_gmn = Ranktraintimecourse-np.transpose(np.tile(subjmean,(5,1)))+grandmean
y = np.mean(y_gmn,0)
sem = np.std(y_gmn,0)/sqrt(y_gmn.shape[0]) # between subject error bars
plt.ylim([1,12])
plt.xlim([0.8,5.2])
plt.xticks(np.arange(1,6))
plt.xlabel('training repetition',fontsize=24)
plt.ylabel('mean target rank',fontsize=24)
plt.tick_params(axis='both', which='major', labelsize=26)
plt.tick_params(axis='both', which='minor', labelsize=26)
plt.tight_layout()
if not os.path.exists('./plots'):
    os.makedirs('./plots')
plt.savefig('./plots/2_training_meanrank_timecourse_blank.pdf')
p = plt.errorbar(x,y,yerr=sem,color=[0.2,0.2,0.2])
plt.savefig('./plots/2_training_meanrank_timecourse.pdf')
plt.close(fig)

def gmn_normalize(dv):    
    y_raw = np.mean(np.transpose(dv),1)
    subjmean = np.mean(np.transpose(dv),0)
    grandmean = np.mean(subjmean)
    y_gmn = dv-np.transpose(np.tile(subjmean,(5,1)))+grandmean
    y = np.mean(y_gmn,0)
    sem = np.std(y_gmn,0)/sqrt(y_gmn.shape[0]) # between subject error bars    
    return y,sem

fig = plt.figure(figsize(4,8))
x = np.arange(1,6,1)
y32, sem32 = gmn_normalize(Top32timecourse)
y16, sem16 = gmn_normalize(Top16timecourse)
y8, sem8 = gmn_normalize(Top8timecourse)
y4, sem4 = gmn_normalize(Top4timecourse)
y1, sem1 = gmn_normalize(Top1timecourse)
plt.axhline(y=(1/64),ls='dashed',color=(0.5,0.5,0.5))
plt.ylim([0.,1.])
plt.xlim([0.8,5.2])
plt.xlabel('training repetition',fontsize=24)
plt.ylabel('proportion target rank in top k',fontsize=24)
plt.xticks(np.arange(1,6))
plt.tick_params(axis='both', which='major', labelsize=26)
plt.tick_params(axis='both', which='minor', labelsize=26)
plt.tight_layout()
if not os.path.exists('./plots'):
    os.makedirs('./plots')
plt.savefig('./plots/2_training_topk_timecourse_blank.pdf')
p = plt.errorbar(x,y1,yerr=sem1,color=[0.,0.,0.],label='top1')
plt.savefig('./plots/2_training_top1_timecourse.pdf')
p = plt.errorbar(x,y32,yerr=sem32,color=[0.8,0.8,0.8],label='top32')
p = plt.errorbar(x,y16,yerr=sem16,color=[0.6,0.6,0.6],label='top16')
p = plt.errorbar(x,y8,yerr=sem8,color=[0.4,0.4,0.4],label='top8')
p = plt.errorbar(x,y4,yerr=sem4,color=[0.2,0.2,0.2],label='top4')
plt.savefig('./plots/2_training_topk_timecourse.pdf')
plt.close(fig)

In [ ]:
## Compute change in target rank by condition as function of drawing training

Rankdifftitc_subj = ah.getSubjMean(Rankdifftitc)
Rankdiffuitc_subj = ah.getSubjMean(Rankdiffuitc)
Rankdiffuiuc_subj = ah.getSubjMean(Rankdiffuiuc)
print '###### Mean change in target rank by condition ######'
print 'Trained: ' + str(np.mean(Rankdifftitc_subj)) + '  sem: '  + str(ah.compute_sem(Rankdifftitc_subj))
print 'Near: ' + str(np.mean(Rankdiffuitc_subj)) + '  sem: '  + str(ah.compute_sem(Rankdiffuitc_subj)) 
print 'Far: ' + str(np.mean(Rankdiffuiuc_subj)) + '  sem: '  + str(ah.compute_sem(Rankdiffuiuc_subj))

print ' '
print '###### SD of change in target rank (post-pre) by condition ######'
print 'Trained: ', np.round(np.std(Rankdifftitc_subj),2), '  Near: ', np.round(np.std(Rankdiffuitc_subj),2), '  Far (subsampled): ', np.round(np.std(Rankdiffuiuc_subj_sub),2)


In [ ]:
# Pairwise comparisons between conditions
import scipy.stats as stats
def boot_mean(data,nIter):
    boot_mean = []
    print '# subjects: ' + str(len(data))
    for n in range(nIter):
        bootgroup = np.random.RandomState(n).choice(data,size=len(data),replace=True) 
        boot_mean.append(np.nanmean(bootgroup))
    boot_mean = map(np.array,[boot_mean]) 
    return boot_mean

def run_ttest(x):
    x = remove_nans(x)
    t,p = stats.ttest_1samp(x,0)
    print 't:', t, '  p:', p
    return t,p
 
def remove_nans(x):
    return x[~np.isnan(x)]

print '---------- Comparisons against zero --------------'
print 'Trained'
t,p = run_ttest(Rankdifftitc_subj)
print 'Near'
t,p = run_ttest(Rankdiffuitc_subj)
print 'Far'
t,p = run_ttest(Rankdiffuiuc_subj)
print '---------- Pairwise comparisons --------------'
print 'Trained vs. Near'
t,p = run_ttest(Rankdifftitc_subj-Rankdiffuitc_subj)
print 'Near vs. Far'
t,p = run_ttest(Rankdiffuitc_subj-Rankdiffuiuc_subj)
print 'Trained vs. Far'
t,p = run_ttest(Rankdifftitc_subj-Rankdiffuiuc_subj)

In [ ]:
## Check for normality: Plotting quantile-quantile plot against normal distribution 
def normalize(x):
    mu = np.mean(x)
    sd = np.std(x)
    return (x-mu)/sd

sns.set_context('talk')
fig = plt.figure(figsize=(15,3))
plt.subplot(1,3,1)
data = normalize(Rankdifftitc_subj)
res = stats.probplot(data,plot=plt)
plt.subplot(1,3,2)
data = normalize(Rankdiffuitc_subj)
res = stats.probplot(data,plot=plt)
plt.subplot(1,3,3)
data = normalize(Rankdiffuiuc_subj)
res = stats.probplot(data,plot=plt)

In [ ]:
### Plot change in rank for each condition vs. 0
X = pd.DataFrame(np.vstack((Rankdifftitc.mean(1),Rankdiffuitc.mean(1),Rankdiffuiuc.mean(1))).transpose())
X.columns = ['trained','near','far']

sns.set_context('poster')
fig, ax = plt.subplots(figsize=(5,5))
sns.barplot(data=X,ci=68,palette='Dark2')
plt.ylim([-2.0,1.0])
plt.xlabel('Condition',fontsize=18)
plt.ylabel('Overall Rank Difference (post-pre)',fontsize=16)
fig.tight_layout()

In [ ]:
## Feature Analyses

## load between-object, within-condition distances ("feature distance", "separation")
between_item_dists_anch = pd.read_csv(os.path.join(data_dir,'between_obj_within_condition_distances.csv'))
between_item_dists_anch = np.vstack((between_item_dists_anch['TITC'].values,\
                                     between_item_dists_anch['UITC'].values,\
                                     between_item_dists_anch['UIUC'].values)).T
TvsF_anch = between_item_dists_anch[:,0]-between_item_dists_anch[:,2]
NvsF_anch = between_item_dists_anch[:,1]-between_item_dists_anch[:,2]

## plot 'em
textsize=20
data = np.vstack((TvsF_anch,NvsF_anch)).T
X = pd.DataFrame(data)
X.columns = ['trained','near']

sns.set_context('poster')
fig = plt.figure(figsize=(2,6))
sns.barplot(data=X,ci=68,palette='Dark2')
plt.ylabel(r'$\Delta$ feature distance (post-pre)', fontsize=textsize)
plt.ylim([-0.02,0.02])
plt.xlabel('Condition',fontsize=18)
plt.axhline(0, color='black',linewidth=0.75)

In [ ]:
# within-item distances ("feature variability" or "sharpening")

within_item_dists = np.load(os.path.join(data_dir,'within_item_distances.npy'))
sns.set_style('white')
sns.set_context('poster')

data = within_item_dists.T
X = pd.DataFrame(data)
X.columns = ['trained']

## plot 'em
textsize=20
sns.set_context('poster')
fig = plt.figure(figsize=(1,6))
sns.barplot(data=X,ci=68,palette='Dark2')
plt.ylabel(r'$\Delta$ feature variability (late-early)',fontsize=textsize)
plt.ylim([-0.08,0.02])
plt.axhline(0, color='black',linewidth=0.75)

In [ ]:
### Static ("Finished drawing control") and Dynamic ("Stroke-by-stroke reconstruction") experiments

## Load in data
# order of columns: 
# ['O_TITC','O_UITC','O_UIUC','D_TITC','D_UITC','D_UIUC','S_TITC','S_UITC','S_UIUC']
percepcontrol = pd.read_csv(os.path.join(data_dir,'Rankdiff_submean_PooledDrawDynamicStatic.csv'))
pc = percepcontrol
o_rdtitc = pc['O_TITC'].values
o_rduitc = pc['O_UITC'].values
o_rduiuc = pc['O_UIUC'].values

d_rdtitc = pc['D_TITC'].values
d_rduitc = pc['D_UITC'].values
d_rduiuc = pc['D_UIUC'].values

s_rdtitc = pc['S_TITC'].values
s_rduitc = pc['S_UITC'].values
s_rduiuc = pc['S_UIUC'].values

rankdiffmat = np.vstack([o_rdtitc,o_rduitc,o_rduiuc,d_rdtitc,d_rduitc,d_rduiuc,s_rdtitc,s_rduitc,s_rduiuc]).T


In [ ]:
## helpers
def compute_sem(a):
    """ return the standard error of the mean """
    return np.std(a,0)/np.sqrt(shape(a)[0])

def compute_gmn(a):
    """ grand-mean-normalize a data matrix """
    y_raw = a
    subjmean = np.mean(a,1)
    grandmean = np.mean(subjmean)
    y_gmn = a-np.transpose(np.tile(subjmean,(shape(a)[1],1)))+grandmean
    m = np.mean(y_gmn,0)
    sem = np.std(y_gmn,0)/np.sqrt(shape(y_gmn)[0])
    return m,sem,y_gmn

def get_boot_CI(data,nIter):
    boot_mean = []
    for n in range(nIter):
        bootgroup = np.random.RandomState(n).choice(data,size=len(data),replace=True) 
        boot_mean.append(np.nanmean(bootgroup))
    boot_mean = map(np.array,[boot_mean]) 
    return np.mean(boot_mean),np.percentile(boot_mean,2.5), np.percentile(boot_mean,97.5)

In [ ]:
print 'Summary of Within-Experiment T-tests'
print ' '
print 'Original Drawing'
print ' '
print 'TITC vs. UIUC (enhancement)'
dv = o_rdtitc-o_rduiuc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))
print 'UITC vs. UIUC (impairment)'
dv = o_rduitc-o_rduiuc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'TITC vs. UITC (trained vs. near)'
dv = o_rdtitc-o_rduitc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))


print 'TITC vs. 0 (trained vs. 0)'
dv = o_rdtitc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'UITC vs. 0 (trained vs. 0)'
dv = o_rduitc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'UIUC vs. 0 (trained vs. 0)'
dv = o_rduiuc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print ' '
print 'Dynamic Viewing'
print 'TITC vs. UIUC (enhancement)'
dv = d_rdtitc-d_rduiuc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'UITC vs. UIUC (impairment)'
dv = d_rduitc-d_rduiuc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'TITC vs. UITC (trained vs. near)'
dv = d_rdtitc-d_rduitc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'TITC vs. 0 (trained vs. 0)'
dv = d_rdtitc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'UITC vs. 0 (near vs. 0)'
dv = d_rduitc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'UIUC vs. 0 (far vs. 0)'
dv = d_rduiuc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))


print ' '
print 'Static Viewing'
print 'TITC vs. UIUC (enhancement)'
dv = s_rdtitc-s_rduiuc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))
print 'UITC vs. UIUC (impairment)'
dv = s_rduitc-s_rduiuc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'TITC vs. UITC (trained vs. near)'
dv = s_rdtitc-s_rduitc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'TITC vs. 0 (trained vs. 0)'
dv = s_rdtitc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'UITC vs. 0 (near vs. 0)'
dv = s_rduitc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'UIUC vs. 0 (far vs. 0)'
dv = s_rduiuc
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print ' '

o_vs_d = o_rdtitc - d_rdtitc
o_vs_s = o_rdtitc - s_rdtitc
d_vs_s = d_rdtitc - s_rdtitc

print 'Drawing Training (o_titc) vs. Dynamic Training (d_titc)'
dv = o_vs_d
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'Drawing Training (o_titc) vs. Static Training (s_titc)'
dv = o_vs_s
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))

print 'Dynamic Training (d_titc) vs. Static Training (s_titc)'
dv = d_vs_s
t,p = stats.ttest_1samp(dv,0)
print 't = ' + str(t) + '  p = ' + str(p) 
print 'mean = ' + str(np.mean(dv)) + '  std = ' + str(np.std(dv))


In [ ]:
# isolate enhancement vs. impairment
# trained category (TITC, UITC) vs. baseline (UIUC)

o_enhance = o_rdtitc-o_rduiuc
o_impair = o_rduitc-o_rduiuc
o_contrast = o_rdtitc-o_rduitc

d_enhance = d_rdtitc-d_rduiuc
d_impair = d_rduitc-d_rduiuc
d_contrast = d_rdtitc-d_rduitc

s_enhance = s_rdtitc-s_rduiuc
s_impair = s_rduitc-s_rduiuc
s_contrast = s_rdtitc-s_rduitc

normalizedrdmat = np.hstack([o_enhance,o_impair,d_enhance,d_impair,s_enhance,s_impair])

onlytrainingmat = np.hstack([o_enhance,d_enhance,s_enhance])

In [ ]:
####### Baseline against Far condition

normalizedrdmat = np.vstack([o_enhance,o_impair,d_enhance,d_impair,s_enhance,s_impair]).T
onlytrainingmat = np.vstack([o_enhance,d_enhance,s_enhance]).T

# Make rank diff summary bar plot from all 3 experiments (N=588 *3)
m = np.mean(normalizedrdmat,0)
sem = compute_sem(normalizedrdmat)
ind = np.arange(len(m[:2]))
width = 0.8
colors = plt.cm.Set1(np.linspace(0, 1, 2))

fig = plt.figure(figsize(10, 6))

plt.subplot(1,3,1)
m,sem,y_gmn = compute_gmn(normalizedrdmat[:,:2])
X = pd.DataFrame(y_gmn)
X.columns = ['trained','near']
sns.barplot(data=X,ci=68)
plt.ylim([-2.0,2.0])
plt.ylabel(r'$\Delta$ target rank (post-pre)',fontsize=20)
plt.title('drawing',fontsize=20)

plt.subplot(1,3,2)
m,sem,y_gmn = compute_gmn(normalizedrdmat[:,2:4])
X = pd.DataFrame(y_gmn)
X.columns = ['trained','near']
sns.barplot(data=X,ci=68)
plt.ylim([-2.0,2.0])
plt.ylabel(r'$\Delta$ target rank (post-pre)',fontsize=20)
plt.title('dynamic',fontsize=20)

plt.subplot(1,3,3)
m,sem,y_gmn = compute_gmn(normalizedrdmat[:,4:6])
X = pd.DataFrame(y_gmn)
X.columns = ['trained','near']
sns.barplot(data=X,ci=68)
plt.ylim([-2.0,2.0])
plt.ylabel(r'$\Delta$ target rank (post-pre)',fontsize=20)
plt.title('static',fontsize=20)

fig.tight_layout()
